# 🎯 YOLO Anti-Cheat Model Fine-tuning

This notebook will fine-tune the existing YOLO model to improve detection of:
- 📱 **Phone** (currently weak - max ~0.2%)
- 📄 **Material/Paper** (currently weak - max ~0.1%)
- 👤 **Person** (needs improvement - max ~5%)
- 🎧 **Headphones** (already good - max ~44%)

## Instructions:
1. Upload your existing `best.pt` model to Google Drive
2. Run all cells in order
3. Download the new `best.onnx` file when done
4. Replace `Intelligence-Test/public/models/anticheat_yolo11s.onnx`

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install dependencies
!pip install ultralytics -q
!pip install onnx onnxruntime -q

import os
import shutil
import yaml
import numpy as np
from pathlib import Path
from ultralytics import YOLO

print("✅ Dependencies installed!")

In [ ]:
# Configuration
TARGET_CLASSES = ['person', 'phone', 'material', 'headphones']

# Dataset URLs from Roboflow (phone and material focused)
DATASETS = [
    # Phone datasets (HIGH PRIORITY - model is weak here)
    ("phone_1", "https://app.roboflow.com/ds/5ReObgnLbQ?key=HTPSgVzDLW"),
    ("phone_2", "https://app.roboflow.com/ds/f9k54F7Azq?key=eYssUekSYc"),
    
    # Paper/Material datasets (HIGH PRIORITY)
    ("paper_1", "https://app.roboflow.com/ds/inuabMtp6t?key=jbu7HTlrBf"),
    ("paper_2", "https://app.roboflow.com/ds/b4oxAhlW40?key=4A761Kjm5F"),
    
    # Headphones (for balance)
    ("headphones_1", "https://app.roboflow.com/ds/qqqEeSKAlk?key=GT1Xa65onI"),
    ("headphones_2", "https://app.roboflow.com/ds/cKHwOqmuda?key=qL10KsWlBt"),
    
    # Person
    ("person_1", "https://app.roboflow.com/ds/PwRwV0c1jL?key=FgXbXeqlpH"),
]

# Class mapping
CLASS_MAPPING = {
    'person': 'person', 'student': 'person', 'face': 'person', 'head': 'person',
    'human': 'person', 'people': 'person', 'man': 'person', 'woman': 'person',
    'phone': 'phone', 'mobile': 'phone', 'cell phone': 'phone', 
    'telephone': 'phone', 'smartphone': 'phone', 'cellphone': 'phone',
    'mobile phone': 'phone', 'iphone': 'phone', 'android': 'phone',
    'paper': 'material', 'document': 'material', 'book': 'material',
    'notebook': 'material', 'notes': 'material', 'sheet': 'material',
    'material': 'material', 'cheat sheet': 'material',
    'headphone': 'headphones', 'headphones': 'headphones', 
    'earphone': 'headphones', 'earphones': 'headphones',
    'headset': 'headphones', 'earbuds': 'headphones', 'earbud': 'headphones',
    'airpods': 'headphones', 'ear device': 'headphones',
}

def normalize_class(class_name):
    class_name = class_name.lower().strip()
    for key, target in CLASS_MAPPING.items():
        if key.lower() == class_name:
            return TARGET_CLASSES.index(target)
    return -1

print("✅ Configuration loaded!")
print(f"Target classes: {TARGET_CLASSES}")

In [ ]:
# Download datasets
!mkdir -p /content/raw_datasets
%cd /content/raw_datasets

for name, url in DATASETS:
    if not os.path.exists(name):
        print(f"📥 Downloading {name}...")
        !mkdir -p {name}
        !curl -L "{url}" > {name}/dataset.zip 2>/dev/null
        !unzip -q {name}/dataset.zip -d {name}
        !rm {name}/dataset.zip
    else:
        print(f"✓ {name} already exists")

print("\n✅ All datasets downloaded!")

In [ ]:
# Merge and convert datasets
OUTPUT_DIR = '/content/merged_dataset'

# Clear and create output directory
!rm -rf {OUTPUT_DIR}
os.makedirs(f"{OUTPUT_DIR}/train/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/train/labels", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/valid/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/valid/labels", exist_ok=True)

total_train = 0
total_valid = 0

for name, _ in DATASETS:
    dataset_dir = f"/content/raw_datasets/{name}"
    if not os.path.exists(dataset_dir):
        continue
    
    # Find data.yaml
    data_yaml = None
    for root, dirs, files in os.walk(dataset_dir):
        if 'data.yaml' in files:
            data_yaml = os.path.join(root, 'data.yaml')
            break
    
    if not data_yaml:
        print(f"⚠️ No data.yaml in {name}")
        continue
    
    with open(data_yaml, 'r') as f:
        config = yaml.safe_load(f)
    
    source_classes = config.get('names', [])
    if isinstance(source_classes, dict):
        source_classes = list(source_classes.values())
    
    print(f"\n📂 Processing {name}...")
    print(f"   Source classes: {source_classes}")
    
    for split in ['train', 'valid', 'test']:
        img_dir = None
        lbl_dir = None
        
        # Try different directory structures
        for try_path in [dataset_dir, os.path.dirname(data_yaml)]:
            if os.path.exists(os.path.join(try_path, split, 'images')):
                img_dir = os.path.join(try_path, split, 'images')
                lbl_dir = os.path.join(try_path, split, 'labels')
                break
        
        if not img_dir or not os.path.exists(img_dir):
            continue
        
        out_split = 'train' if split in ['train', 'test'] else 'valid'
        count = 0
        
        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            # Copy image
            src_img = os.path.join(img_dir, img_file)
            dst_img = os.path.join(OUTPUT_DIR, out_split, 'images', f"{name}_{img_file}")
            shutil.copy(src_img, dst_img)
            
            # Convert label
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            src_lbl = os.path.join(lbl_dir, lbl_file)
            dst_lbl = os.path.join(OUTPUT_DIR, out_split, 'labels', f"{name}_{lbl_file}")
            
            if os.path.exists(src_lbl):
                with open(src_lbl, 'r') as f:
                    lines = f.readlines()
                
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    
                    old_class_id = int(parts[0])
                    if old_class_id < len(source_classes):
                        old_class_name = source_classes[old_class_id]
                        new_class_id = normalize_class(old_class_name)
                        
                        if new_class_id >= 0:
                            parts[0] = str(new_class_id)
                            new_lines.append(' '.join(parts))
                
                if new_lines:
                    with open(dst_lbl, 'w') as f:
                        f.write('\n'.join(new_lines))
                    count += 1
        
        if out_split == 'train':
            total_train += count
        else:
            total_valid += count
        
        if count > 0:
            print(f"   {split} -> {out_split}: {count} images")

# Create data.yaml
data_yaml_content = {
    'path': OUTPUT_DIR,
    'train': 'train/images',
    'val': 'valid/images',
    'names': {i: name for i, name in enumerate(TARGET_CLASSES)},
    'nc': len(TARGET_CLASSES),
}

with open(f"{OUTPUT_DIR}/data.yaml", 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print(f"\n" + "="*50)
print(f"✅ Dataset prepared!")
print(f"   Train images: {total_train}")
print(f"   Valid images: {total_valid}")
print(f"   Classes: {TARGET_CLASSES}")
print(f"="*50)

In [ ]:
# Upload or specify base model path
# Option 1: Upload from local machine
# from google.colab import files
# uploaded = files.upload()  # Upload best.pt
# BASE_MODEL_PATH = list(uploaded.keys())[0]

# Option 2: Use from Google Drive (recommended)
BASE_MODEL_PATH = '/content/drive/MyDrive/Intelligence-Test-Models/anticheat_objects_v2_headphones/weights/best.pt'

# Option 3: Start from pretrained YOLO11
# BASE_MODEL_PATH = 'yolo11s.pt'

print(f"Base model: {BASE_MODEL_PATH}")
if os.path.exists(BASE_MODEL_PATH):
    print("✅ Model file found!")
else:
    print("❌ Model file not found! Please check path.")

In [ ]:
# Fine-tune the model
model = YOLO(BASE_MODEL_PATH)

print("🚀 Starting fine-tuning...")
print("This may take 30-60 minutes depending on dataset size.")

results = model.train(
    data=f"{OUTPUT_DIR}/data.yaml",
    epochs=50,              # Number of epochs
    imgsz=640,              # Image size
    batch=16,               # Batch size (reduce if OOM)
    patience=15,            # Early stopping
    lr0=0.001,              # Lower LR for fine-tuning
    lrf=0.01,               # Final LR factor
    warmup_epochs=3,        # Warmup
    freeze=10,              # Freeze first 10 layers
    project='/content/drive/MyDrive/Intelligence-Test-Models',
    name='anticheat_finetuned_v3',
    exist_ok=True,
    device=0,               # GPU
    verbose=True,
)

print("\n✅ Training completed!")

In [ ]:
# Export to ONNX
BEST_MODEL_PATH = '/content/drive/MyDrive/Intelligence-Test-Models/anticheat_finetuned_v3/weights/best.pt'

model = YOLO(BEST_MODEL_PATH)

print("📦 Exporting to ONNX...")
model.export(
    format='onnx',
    imgsz=640,
    simplify=True,
    dynamic=False,
    opset=17
)

onnx_path = BEST_MODEL_PATH.replace('.pt', '.onnx')
print(f"\n✅ ONNX model saved to: {onnx_path}")

In [ ]:
# Test the new model
import onnxruntime as ort
from PIL import Image, ImageDraw

session = ort.InferenceSession(onnx_path)
input_name = session.get_inputs()[0].name
classes = ['person', 'phone', 'material', 'headphones']

def test_image(name, img):
    img = img.convert('RGB').resize((640, 640))
    img_array = np.array(img).astype(np.float32) / 255.0
    img_array = np.transpose(img_array, (2, 0, 1))
    img_array = np.expand_dims(img_array, axis=0)
    
    outputs = session.run(None, {input_name: img_array})
    output = outputs[0]
    class_scores = output[0, 4:8, :]
    
    print(f"\n{name}:")
    for i, cls in enumerate(classes):
        max_score = class_scores[i].max()
        print(f"  {cls}: max={max_score:.4f}")

# Test with phone simulation
phone_img = Image.new('RGB', (640, 640), color=(200, 200, 200))
draw = ImageDraw.Draw(phone_img)
draw.rectangle([250, 200, 390, 450], fill=(20, 20, 20))
draw.rectangle([260, 210, 380, 440], fill=(50, 50, 80))
test_image("Phone simulation", phone_img)

# Test with headphones simulation
headphones_img = Image.new('RGB', (640, 640), color=(200, 200, 200))
draw = ImageDraw.Draw(headphones_img)
draw.arc([200, 150, 440, 350], 180, 0, fill=(30, 30, 30), width=20)
draw.ellipse([180, 280, 260, 400], fill=(40, 40, 40))
draw.ellipse([380, 280, 460, 400], fill=(40, 40, 40))
test_image("Headphones simulation", headphones_img)

print("\n" + "="*50)
print("If phone scores improved (>0.1), the fine-tuning worked!")
print("="*50)

In [ ]:
# Download the ONNX file
print("\n📋 NEXT STEPS:")
print("1. Download the ONNX file from Google Drive")
print(f"   Location: {onnx_path}")
print("2. Rename it to: anticheat_yolo11s.onnx")
print("3. Copy to: Intelligence-Test/public/models/")
print("4. Rebuild and deploy the web app")
print("\n🎉 Done!")